### 1. Preprocessing

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from preprocessing import combine_data_frompath
import datetime
from tensorflow.keras import callbacks



run this code below to see that you are using GPU for computing

In [ ]:
tf.config.list_physical_devices("GPU")

In [ ]:
train_data = combine_data_frompath('aug_data\\train\\labels\\*.json','aug_data\\train\\images\\*.jpg')
val_data = combine_data_frompath('aug_data\\val\\labels\\*.json','aug_data\\val\\images\\*.jpg')

### 2. Create Model

In [ ]:
from model import FaceTracker
facetracker = FaceTracker()

In [ ]:
batches_per_epoch = len(train_data)
lr_decay = (1./0.75 -1)/batches_per_epoch
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)
facetracker.compile(optimizer=opt)

### 3. Train Model

## 3.1 Create Callback

In [ ]:
# this is for the tensorboard we will plot the training loss in tensorboard
time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/training/" + time 
tf_board_callback = callbacks.TensorBoard(log_dir=logdir)
early_stopping_callback = callbacks.EarlyStopping(monitor="val_total_loss",patience=3)
num_epochs = 20

## 3.2 Run Training 

In [ ]:
facetracker.fit(train_data,epochs=num_epochs,
                callbacks=[
                            tf_board_callback,
                            early_stopping_callback],
                validation_data=val_data,
                verbose=1)

save the model

In [ ]:
import os 
path = "face_tracker_model"
version = "00001"
facetracker.save(filepath=os.path.join(path,version),save_format="tf")

docker run -p 8501:8501 --name face_tracker --moun type=bind,source=I:\github_LP\face_detection\face_tracker_model\,target=/model/facetracker -e MODEL_NAME=face_tracker -t tensorflow/serving

In [ ]:
docker run -p 8501:8501 --name tfserving_facetracker --mount type=bind,source=I:\github_LP\face_detection\facetracker\,target=/models/facetracker -e MODEL_NAME=facetracker -t tensorflow/serving

### 4. Test Model

In [ ]:
facetracker.load_weights("facetracker\\00001")

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### 5. Serving

In [16]:
model_name = "facetracker"
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

facetracker/
    00001/
        keras_metadata.pb
        saved_model.pb
        assets/
        variables/
            variables.data-00000-of-00001
            variables.index


first install docker then 
docker pull tensorflow/serving
then run the following command

docker run -p 8501:8501 --name tfserving_facetracker --mount type=bind,source=[model_source],target=/models/facetracker -e MODEL_NAME=facetracker -t tensorflow/serving

note that model_source is the the location where you save your model for example

source = "...\facetracker\"
... denotes the path to the folder


In [1]:
import requests
import json
import cv2
import numpy as np
url = 'http://localhost:8501/v1/models/facetracker:predict'

def preprocess_input(image):
   img = cv2.resize(image,[120,120])/255.0
   return img[np.newaxis]

def make_prediction(instances):
   img = preprocess_input(instances)
   data = json.dumps({"signature_name": "serving_default", "instances": img.tolist()})
   headers = {"content-type": "application/json"}
   json_response = requests.post(url, data=data, headers=headers)
   predictions = json.loads(json_response.text)['predictions']
   return predictions

In [15]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    
    yhat = make_prediction(rgb)
    sample_coords = yhat[0]["output_2"]
    
    if yhat[0]["output_1"][0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print(yhat)
        break
cap.release()
cv2.destroyAllWindows()

[{'output_1': [0.999990463], 'output_2': [0.413297325, 0.254090399, 0.840602636, 0.710921168]}]
